* At very high costs, deployments would be come very expensive and hardening would be better even for prep_enough
* At what frequency level, it would become sensible to harden than to protect

* make a plot of substations which are flooded with max height of more than prep_level and have highest frequency of such flooding.


* For prep_enough, preparedness always better than hardening.

## Heuristic process:

* Only filter flooded substations, for rest set corresponding x,y,z to fixed values
* For the 72 flooded substations,
    * If max_flood <= max_prep:
        * These (24 substations) can be protected from just tiger dams, so set their x,y = 0
    * For rest of (72-24 = 48 substations):
        * Compute hardening cost
      
    

In [61]:
%matplotlib inline

import yaml
from utils import *
import pandas as pd
pd.set_option('display.max_rows',500)

with open(r'multi.yaml') as file:
    params = yaml.load(file, Loader=yaml.FullLoader)

params["td_cost"] = 40000
params["voll"] = 6000

In [62]:
def get_df_for_heuristic():
    df = pd.read_csv("/home1/07346/ashukla/ThreeStageModel/data/192_Scenario/Final_Input1.csv")
    directions = ["w", "wnw", "nw", "nnw", "n", "nne"]
    categories = ["2", "3", "4", "5"]
    forward_speeds = ["05", "10", "15", "25"]
    lister = []
    for i in directions:
        for j in range(len(categories)):
            for k in range(len(forward_speeds)):
                lister.append("max_flood_level_" + i +"_" + categories[j] + "_" + forward_speeds[k])
    df = df[list(df.columns[0:9]) + lister]
    df_sub = df[["SubNum", "load"]].groupby("SubNum").sum()
    df_flood = df[["SubNum"] + lister]
    df_flood = df_flood.drop_duplicates().set_index("SubNum") # drop duplicates
    df_flood = df_flood.loc[(df_flood.sum(axis=1) != 0), :] # drop substations that are not flooded
    """df_sub has load demand for all the substations"""
    """df_flood has only flooded substations"""
    return df, df_sub, df_flood

def flood_info_generator(model_scenarios, df_flood):
    """Dictionary flood_info contains information on how frequently was each substation flooded 
    in each mini-Brent case. They key is substation id. The value is another dictionary whose each 
    entry is mini-Brent model id. And the value of this dictionary is how many scenarios 
    within this mini-brent model was that substation flooded."""
    flood_info = {}
    for i in df_flood.index:
        flood_info[i] = {}
        for j in model_scenarios:
            counter = 0
            for k in model_scenarios[j]:
                if df_flood.loc[i, k] > 0:
                    counter = counter + 1
            flood_info[i][j] = counter
    return flood_info

def compute_load_shed_no_measure(flood_info, df_sub, model_scenarios, params, df_flood):
    expected_load_shed = {}
    for i in flood_info:
        load_value = df_sub.loc[i,"load"]
        temp = 0
        for j in flood_info[i]:
            """We see in how many leaves out of 216 it is flooded"""
            temp = temp + load_value*flood_info[i][j]
        expected_load_shed[i] = temp/(len(model_scenarios.keys())*len(model_scenarios[0]))
        expected_load_shed[i] = [round(expected_load_shed[i]*params["voll"]*params["restore_time"]*params["tau"]/1e6,3), df_flood.loc[i,:].max()]
    main_df = pd.DataFrame(expected_load_shed).T
    main_df.columns = ["voll_million", "max_flood"]
    main_df["prepare_million"] = 1000
    main_df["mitigate_million"] = 1000
    """Every substation in main_df is flooded atleast once"""
    for i in main_df.index:
        hardening_cost = params["fixed_cost"] + params["variable_cost"]*main_df.loc[i, "max_flood"]
        main_df.loc[i, "mitigate_million"] = round(hardening_cost/1e6, 3)
    return main_df

def prep_cost_computer(prep_enough, flood_info, params, model_scenarios):
    for i in prep_enough.index:
        temp = 0 # temp is counter of number of mini-brent cases when tiger dams need to be deployed
        for j in flood_info[i]:
            if flood_info[i][j] > 0:
                temp = temp + 1
        temp = temp/len(model_scenarios.keys())
        temp = temp*params["operating_cost"]*params["tau"]
        prep_enough.loc[i, "prepare_million"] = round((prep_enough.loc[i, "max_flood"]*params["td_cost"] + temp)/1e6, 3)
    
    return prep_enough

def mit_prep_fixable_substation(mit_enough, df_flood, params):
    temp_df = df_flood.loc[mit_enough.index,:].copy()
    temp_df = temp_df[(temp_df > 0) & (temp_df <= params["prep_level"])]
    temp_df = temp_df.dropna(thresh=1)
    can_be_fixed = list(temp_df.index)
    return can_be_fixed

def mit_cost_computer(mit_enough, params, flood_info, model_scenarios, df_flood, df_sub):
    """For each substation deploy_count counts the number of times, deployment happened in mini-brent models. 
    load_loss counts the number of times flooding happened"""
    deploy_count = {} 
    load_loss = {}
    for i in mit_enough.index:
        deploy_count[i] = 0
        load_loss[i] = 0
        max_preventable_flooding = 0
        for j in flood_info[i]:
            flag = 0
            for k in model_scenarios[j]:
                flood_level = df_flood.loc[i,k]
                if (flood_level > 0) & (flood_level <= params["prep_level"]):
                    flag = 1
                    if flood_level > max_preventable_flooding:
                        max_preventable_flooding = flood_level
                if flood_level > params["prep_level"]:
                    load_loss[i] = load_loss[i] + 1
            deploy_count[i] = deploy_count[i] + flag
        deploy_count[i] = [deploy_count[i], max_preventable_flooding]
    for i in deploy_count:
        deploy_count[i].append(load_loss[i])
    mitigate = pd.DataFrame(deploy_count).T
    mitigate.columns = ["deploy_counts", "max_td_units", "load_loss_flood_counts"]
    
    for i in mitigate.index:
        O_C = (mitigate.loc[i,"deploy_counts"]/len(model_scenarios.keys()))*params["operating_cost"]*params["tau"]
        V_C = mitigate.loc[i,"load_loss_flood_counts"]/(len(model_scenarios.keys())*len(model_scenarios[0]))
        V_C = V_C*df_sub.loc[i,"load"]*params["voll"]*params["restore_time"]*params["tau"]
        A_C = params["td_cost"]*mitigate.loc[i,"max_td_units"]
        T_C = O_C + V_C + A_C
        T_C = round(T_C/1e6,3)
        mit_enough.loc[i, "prepare_million"] = T_C    
    return mitigate, mit_enough

def heuristic(df_type, flood_info, df_flood, model_scenarios, params):
    heuristic_dictionary = {}
    for i in df_type.index:
        heuristic_dictionary[i] = {}
        min_value = min(df_type.loc[i,"voll_million"], 
                        min(df_type.loc[i,"prepare_million"], df_type.loc[i,"mitigate_million"]))
        if min_value == df_type.loc[i,"prepare_million"]:
            heuristic_dictionary[i]["x_mit"] = 0
            heuristic_dictionary[i]["y_mit"] = 0
            for j in flood_info[i]:
                if flood_info[i][j] > 0:
                    flag = 0
                    max_preventable_flooding = 0
                    for k in model_scenarios[j]:
                        flood_level = df_flood.loc[i,k]
                        if (flood_level > 0) & (flood_level <= params["prep_level"]):
                            flag = 1
                            if flood_level > max_preventable_flooding:
                                max_preventable_flooding = flood_level
                    if flag == 1:
                        heuristic_dictionary[i]["p_" + str(j)] = max_preventable_flooding
                        heuristic_dictionary[i]["q_" + str(j)] = 1
                    else:
                        heuristic_dictionary[i]["p_" + str(j)] = 0
                        heuristic_dictionary[i]["q_" + str(j)] = 0   
                else:
                    heuristic_dictionary[i]["p_" + str(j)] = 0
                    heuristic_dictionary[i]["q_" + str(j)] = 0  
        elif min_value == df_type.loc[i,"mitigate_million"]:
            heuristic_dictionary[i]["x_mit"] = df_type.loc[i, "max_flood"]
            heuristic_dictionary[i]["y_mit"] = 1
            for j in flood_info[i]:  
                heuristic_dictionary[i]["p_" + str(j)] = 0
                heuristic_dictionary[i]["q_" + str(j)] = 0
        else:
            heuristic_dictionary[i]["x_mit"] = 0
            heuristic_dictionary[i]["y_mit"] = 0
            for j in flood_info[i]:
                heuristic_dictionary[i]["p_" + str(j)] = 0
                heuristic_dictionary[i]["q_" + str(j)] = 0
    return heuristic_dictionary

def tighten_model(df_sub, df_flood, flood_info, model_scenarios):
    heuristic_dictionary = {}
    flood_list = list(df_flood.index)
    for i in df_sub.index:
        if i not in flood_list:
            heuristic_dictionary[i] = {}
            heuristic_dictionary[i]["x_mit"] = 0
            heuristic_dictionary[i]["y_mit"] = 0
            for j in model_scenarios:
                heuristic_dictionary[i]["p_" + str(j)] = 0
                heuristic_dictionary[i]["q_" + str(j)] = 0
        else:
            continue
    return heuristic_dictionary

## Heuristic_Development

In [63]:
model_scenarios = return_model_scenarios()
df, df_sub, df_flood = get_df_for_heuristic()
flood_info = flood_info_generator(model_scenarios, df_flood)
main_df = compute_load_shed_no_measure(flood_info, df_sub, model_scenarios, params, df_flood)

prep_enough = main_df[(main_df["max_flood"] <= params["prep_level"])].copy()
mit_enough = main_df[(main_df["max_flood"] > params["prep_level"])].copy()
prep_enough = prep_enough[['voll_million', 'prepare_million', 'mitigate_million', 'max_flood']]
mit_enough = mit_enough[['voll_million', 'prepare_million', 'mitigate_million', 'max_flood']]

prep_enough = prep_cost_computer(prep_enough, flood_info, params, model_scenarios)
mitigate, mit_enough = mit_cost_computer(mit_enough, params, flood_info, model_scenarios, df_flood, df_sub)

prep_dict = heuristic(prep_enough, flood_info, df_flood, model_scenarios, params)
mit_dict = heuristic(mit_enough, flood_info, df_flood, model_scenarios, params)
rest_dict = tighten_model(df_sub, df_flood, flood_info, model_scenarios)

## Knapsacking in the presence of budget constraint

In [64]:
import os
import pandas as pd
import json
from utils import *
from three_stage_model import *

main_path = "/work2/07346/ashukla/stampede2/ThreeStageModel/output/modified_td_voll_" + str(params["voll"]) + "/"

with open(main_path + "model_params.json", 'r') as f:
    params = json.load(f)        
with open(main_path + "model_scenarios.json", 'r') as f:
    model_scenarios_string = json.load(f)

params["path_to_input"] = os.getcwd() + "/data/192_Scenario/"
base_model = three_stage_model(params, model_scenarios)
base_model.model.update()
sol_path = main_path + "solution.sol"
base_model.model.read(sol_path)
base_model.model.update()

In [65]:
lister = []
for sub_id in df_sub.index:
    t1 = base_model.model.getVarByName('x_mit[' +  str(sub_id) + ']').Start
    t2 = base_model.model.getVarByName('y_mit[' +  str(sub_id) + ']').Start
    lister.append([t1,t2])

In [66]:
mit_dict[247]['x_mit']

11.0

In [53]:
t1 = pd.DataFrame(lister).set_index(df_sub.index)
t1 = t1[t1[0] > 0]

In [56]:
for i in t1.index:
    if i in mit_dict:
        print(i, t1.loc[i,0], mit_dict[i]['x_mit'])
    else:
        if t1.loc[i,0] > 0:
            print("issue", i, t1.loc[i,0] )

301 13.0 13.0
334 13.0 0
923 10.0 10.0
932 18.0 18.0
953 21.0 21.0
954 21.0 21.0
955 12.0 12.0
956 12.0 0
957 14.0 14.0
973 14.0 14.0
984 19.0 19.0
1016 14.0 14.0
1062 21.0 21.0
1069 8.0 8.0
1070 12.0 12.0
1107 17.0 17.0
1111 19.0 19.0
1132 21.0 21.0
1145 21.0 21.0


In [59]:
for i in 

{'x_mit': 0,
 'y_mit': 0,
 'p_0': 3,
 'q_0': 1,
 'p_1': 0,
 'q_1': 0,
 'p_2': 0,
 'q_2': 0,
 'p_3': 3,
 'q_3': 1,
 'p_4': 5,
 'q_4': 1,
 'p_5': 5,
 'q_5': 1,
 'p_6': 0,
 'q_6': 0,
 'p_7': 5,
 'q_7': 1,
 'p_8': 5,
 'q_8': 1,
 'p_9': 1,
 'q_9': 1,
 'p_10': 0,
 'q_10': 0,
 'p_11': 0,
 'q_11': 0,
 'p_12': 1,
 'q_12': 1,
 'p_13': 5,
 'q_13': 1,
 'p_14': 6,
 'q_14': 1,
 'p_15': 0,
 'q_15': 0,
 'p_16': 5,
 'q_16': 1,
 'p_17': 6,
 'q_17': 1,
 'p_18': 0,
 'q_18': 0,
 'p_19': 0,
 'q_19': 0,
 'p_20': 0,
 'q_20': 0,
 'p_21': 5,
 'q_21': 1,
 'p_22': 4,
 'q_22': 1,
 'p_23': 4,
 'q_23': 1,
 'p_24': 5,
 'q_24': 1,
 'p_25': 4,
 'q_25': 1,
 'p_26': 4,
 'q_26': 1,
 'p_27': 0,
 'q_27': 0,
 'p_28': 0,
 'q_28': 0,
 'p_29': 0,
 'q_29': 0,
 'p_30': 3,
 'q_30': 1,
 'p_31': 1,
 'q_31': 1,
 'p_32': 0,
 'q_32': 0,
 'p_33': 5,
 'q_33': 1,
 'p_34': 5,
 'q_34': 1,
 'p_35': 6,
 'q_35': 1,
 'p_36': 0,
 'q_36': 0,
 'p_37': 0,
 'q_37': 0,
 'p_38': 0,
 'q_38': 0,
 'p_39': 1,
 'q_39': 1,
 'p_40': 0,
 'q_40': 0,
 'p_41': 0

In [60]:
df_flood.loc[334,:]

max_flood_level_w_2_05       0
max_flood_level_w_2_10       0
max_flood_level_w_2_15       0
max_flood_level_w_2_25       0
max_flood_level_w_3_05       3
max_flood_level_w_3_10       0
max_flood_level_w_3_15       0
max_flood_level_w_3_25       0
max_flood_level_w_4_05       9
max_flood_level_w_4_10       8
max_flood_level_w_4_15       5
max_flood_level_w_4_25       5
max_flood_level_w_5_05      13
max_flood_level_w_5_10      13
max_flood_level_w_5_15      11
max_flood_level_w_5_25      12
max_flood_level_wnw_2_05     0
max_flood_level_wnw_2_10     0
max_flood_level_wnw_2_15     0
max_flood_level_wnw_2_25     0
max_flood_level_wnw_3_05     1
max_flood_level_wnw_3_10     0
max_flood_level_wnw_3_15     0
max_flood_level_wnw_3_25     0
max_flood_level_wnw_4_05     7
max_flood_level_wnw_4_10     7
max_flood_level_wnw_4_15     5
max_flood_level_wnw_4_25     6
max_flood_level_wnw_5_05    11
max_flood_level_wnw_5_10    11
max_flood_level_wnw_5_15    11
max_flood_level_wnw_5_25    12
max_floo

In [57]:
mit_enough

,voll_million,prepare_million,mitigate_million,max_flood
247,2.488,0.729,1.125,11.0
274,1.334,0.947,1.825,18.0
288,0.099,0.328,1.425,14.0
297,2.295,0.667,1.125,11.0
299,0.850,0.375,1.025,10.0
301,5.926,2.239,1.325,13.0
311,0.603,0.263,0.725,7.0
327,0.545,0.251,0.725,7.0
329,0.000,0.213,0.925,9.0
332,1.140,0.317,0.725,7.0
